In [1]:
import matplotlib.pyplot as plt
import numpy as np

from mnist import MNIST
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D, Dense, Flatten, BatchNormalization, ReLU, Reshape, Lambda
import tensorflow.keras.backend as K

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [2]:
# open data
mndata = MNIST('./python-mnist/data/')
images, labels = mndata.load_training()

# convert to numpy
images = np.array(images)
labels = np.array(labels)

# transform
images = images.reshape(-1,28,28)#.astype(float)/255
b = np.zeros((labels.size, labels.max()+1))
b[np.arange(labels.size),labels] = 1
labels = b

# check
images.shape, labels.shape

((60000, 28, 28), (60000, 10))

In [3]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)

# Model

In [4]:
model = Sequential()

model.add(Reshape((28,28,1), input_shape=(28,28)))
model.add(Lambda(lambda x: K.cast(x, "float32")/255))
model.add(Conv2D(2,kernel_size=(2,2)))
#model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(4,kernel_size=(2,2)))
#model.add(BatchNormalization())
model.add(ReLU())
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(4))
#model.add(BatchNormalization())
model.add(ReLU())
model.add(Dense(10, activation='sigmoid'))

opt = Adam(learning_rate=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
lambda (Lambda)              (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 27, 27, 2)         10        
_________________________________________________________________
re_lu (ReLU)                 (None, 27, 27, 2)         0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 2)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 12, 4)         36        
_________________________________________________________________
re_lu_1 (ReLU)               (None, 12, 12, 4)         0

# Training

In [ ]:
model.fit(X_train,y_train, epochs=8, batch_size=16, validation_data=(X_test,y_test))

Train on 54000 samples, validate on 6000 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/8
54000/54000 [==============================] - 17s 316us/sample - loss: 2.0537 - acc: 0.1911 - val_loss: 1.7242 - val_acc: 0.2773
Epoch 2/8
54000/54000 [==============================] - 16s 305us/sample - loss: 1.4981 - acc: 0.4011 - val_loss: 1.3614 - val_acc: 0.5010
Epoch 3/8
54000/54000 [==============================] - 18s 335us/sample - loss: 1.2899 - acc: 0.5363 - val_loss: 1.2540 - val_acc: 0.5475
Epoch 4/8
54000/54000 [==============================] - 19s 346us/sample - loss: 1.2105 - acc: 0.5721 - val_loss: 1.1980 - val_acc: 0.5797
Epoch 5/8
54000/54000 [==============================] - 17s 317us/sample - loss: 1.1616 - acc: 0.5958 - val_loss: 1.1474 - val_acc: 0.6050
Epoch 6/8
31440/54000 [================>.............] - ETA: 6s - loss: 1.1361 - acc: 0.6096

## Original acc

In [ ]:
preds = model.predict(X_test)

In [ ]:
res = np.argmax(preds,axis=1) == np.argmax(y_test,axis=1)
np.sum(res)/len(res)

# Save

In [ ]:
model.save("model.h5")

# Convert to TF format

In [ ]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    """
    Freezes the state of a session into a pruned computation graph.

    Creates a new computation graph where variable nodes are replaced by
    constants taking their current value in the session. The new graph will be
    pruned so subgraphs that are not necessary to compute the requested
    outputs are removed.
    @param session The TensorFlow session to be frozen.
    @param keep_var_names A list of variable names that should not be frozen,
                          or None to freeze all the variables in the graph.
    @param output_names Names of the relevant graph outputs.
    @param clear_devices Remove the device directives from the graph for better portability.
    @return The frozen graph definition.
    """
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = tf.graph_util.convert_variables_to_constants(
            session, input_graph_def, output_names, freeze_var_names)
        return frozen_graph

In [ ]:
k_inputs = [out.op.name for out in model.inputs]
k_outputs = [out.op.name for out in model.outputs]
k_inputs, k_outputs

In [ ]:
frozen_graph = freeze_session(K.get_session(), output_names=k_outputs)

In [ ]:
tf.train.write_graph(frozen_graph, "tf_model", "my_model.pb", as_text=False)

# TODO: To TRT

In [ ]:
graph = tf.Graph()
sess = tf.InteractiveSession(graph = graph)

with tf.gfile.GFile("./tf_model/my_model.pb", 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())

In [ ]:
from tensorflow.python.compiler.tensorrt import trt_convert as trt

In [ ]:
converter = trt.TrtGraphConverter(
            input_graph_def=graph_def,
            nodes_blacklist=k_outputs,
            max_workspace_size_bytes=1 << 32,
            precision_mode='INT8',
            minimum_segment_size=2,
            is_dynamic_op=True,
            maximum_cached_engines=100)
trt_frozen_graph = converter.convert()

In [ ]:
type(trt_frozen_graph)